**Confronto tra algoritmo genetico e held karp per il problema del commesso viaggiatore generando casualmente un grafo**

In [1]:
import networkx as nx
import random
import sys
import pandas as pd
sys.path.append('..')

from GeneticAlgorithm import GeneticAlgorithm
from HeldKarp import HeldKarp

**Parametri del grafo e generazione casuale**

In [2]:
width = 10
height = 10
num_obstacles = 20
num_gold = 10
num_simulation = 100

In [3]:
def generate_random_grid_graph(width, height, num_obstacles):
    G = nx.grid_2d_graph(width, height)

    # Aggiungi connessioni diagonali
    for i in range(width - 1):
        for j in range(height - 1):
            if (i, j) in G and (i + 1, j + 1) in G:
                G.add_edge((i, j), (i + 1, j + 1))
            if (i + 1, j) in G and (i, j + 1) in G:
                G.add_edge((i + 1, j), (i, j + 1))

    # Aggiungi ostacoli rimuovendo nodi casuali
    obstacles = 0
    while obstacles < num_obstacles:
        x = random.randint(0, width - 1)
        y = random.randint(0, height - 1)
        if (x, y) in G:
            G.remove_node((x, y))
            obstacles += 1

    return G

def place_gold_positions(G, num_gold):
    """
    Posizionamneto casuale dell'oro sul grafo
    """
    gold_positions = []
    available_nodes = list(G.nodes)
    while len(gold_positions) < num_gold:
        node = random.choice(available_nodes)
        if node not in gold_positions:
            gold_positions.append(node)
    return gold_positions

**Simulazione algoritmo genetico e HeldKarp con num_simulation grafici**

In [4]:
import time

# Inizializzazione delle liste per i risultati
results_genetic = []
results_heldkarp = []
execution_times_genetic = []
execution_times_heldkarp = []

num_simulation = 10

for i in range(num_simulation):
    # Genera il grafo e posiziona l'oro
    world_graph = generate_random_grid_graph(width, height, num_obstacles)
    gold_positions = place_gold_positions(world_graph, num_gold)

    # Seleziona casualmente la posizione di partenza tra i nodi disponibili
    available_nodes = [node for node in world_graph.nodes if node not in gold_positions]
    starting_position = random.choice(available_nodes)

    try:
        # Misurazione del tempo di esecuzione dell'algoritmo genetico
        start_time_genetic = time.time()
        optimizer = GeneticAlgorithm(world_graph, gold_positions, starting_position=starting_position)
        _, best_distance_genetic = optimizer.optimize(population_size=1000, num_generations=10)
        end_time_genetic = time.time()
        
        execution_time_genetic = end_time_genetic - start_time_genetic


        # Misurazione del tempo di esecuzione dell'algoritmo Held-Karp
        start_time_heldkarp = time.time()
        heldKarp = HeldKarp(world_graph, gold_positions, starting_position)
        best_distance_heldKarp = heldKarp.solve_tsp()
        end_time_heldkarp = time.time()

        execution_time_heldkarp = end_time_heldkarp - start_time_heldkarp

        # Salvataggio dei risultati
        results_genetic.append(best_distance_genetic)
        results_heldkarp.append(best_distance_heldKarp)
        execution_times_genetic.append(execution_time_genetic)
        execution_times_heldkarp.append(execution_time_heldkarp)
        print(f"Iterazione {i}")
        
    except Exception as e:
        print("Errore:", e)

# Output dei risultati finali
results = pd.DataFrame({
    "genetic": results_genetic, 
    "heldkarp": results_heldkarp,
    "time_genetic": execution_times_genetic,
    "time_heldkarp": execution_times_heldkarp
})

results["diff"] = results["genetic"] - results["heldkarp"]
results["growth_rate"] = results["diff"] / results["heldkarp"] * 100
results


Iterazione 0


/home/kamer/miniconda3/envs/nle/lib/python3.8/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/kamer/miniconda3/envs/nle/lib/python3.8/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Iterazione 1
Iterazione 2
Iterazione 3
Iterazione 4
Iterazione 5
Iterazione 6
Iterazione 7
Iterazione 8
Iterazione 9


,genetic,heldkarp,time_genetic,time_heldkarp,diff,growth_rate
0,29,28,4.838830,0.023606,1,3.571429
1,24,23,3.103034,0.013715,1,4.347826
2,36,35,3.517029,0.014701,1,2.857143
3,27,25,2.873098,0.013999,2,8.000000
4,27,25,2.872879,0.013824,2,8.000000
5,33,32,2.851473,0.021925,1,3.125000
6,31,22,2.855476,0.013935,9,40.909091
7,29,27,2.724672,0.014705,2,7.407407
8,28,28,2.639307,0.013979,0,0.000000
9,29,28,2.978867,0.016717,1,3.571429


In [5]:
results[["diff", "growth_rate"]].describe()

,diff,growth_rate
count,10.000000,10.000000
mean,2.000000,8.178932
std,2.538591,11.784249
min,0.000000,0.000000
25%,1.000000,3.236607
50%,1.000000,3.959627
75%,2.000000,7.851852
max,9.000000,40.909091
